# Segment raw images to get gray matter before normalising.
This is to prevent errors in normalisation down the line (e.g. enlargement of ventricles: GM WM growth or shrinkage?)

Gather necessary pre-requisites:

In [1]:
# import everything you need
from nipype import Node, Workflow
import nipype.interfaces.spm as spm
import nibabel as nb
import numpy as np
import pandas as pd
import os, re, random
from itertools import zip_longest

200303-21:50:46,578 nipype.utils INFO:
	 Running nipype version 1.5.0-dev (latest: 1.4.2)


In [2]:
# define useful functions

# get subject directories from parent directory
def get_subdirs(dir):
    subdirs=[]
    for i in os.listdir(dir):
        # verify subject number in directory and exclude other dirs
        match=re.match(r'(.*\d{3}$)', i)
        if os.path.isdir(os.path.join(dir, i)) and match:
            subdirs.append(os.path.join(dir, match.group(0)))
    return sorted(subdirs)

Get files

In [3]:
# set VBM analysis directory
VBM_dir = "/VBM"

# get FEDXXX dirs
subdirs=get_subdirs(VBM_dir)
print(subdirs)

['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED011', '/VBM/FED012', '/VBM/FED013', '/VBM/FED014', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED018', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED049', '/VBM/FED050', '/VBM/FED051', '/VBM/FED052', '/VBM/FED053', '/VBM/FED054', '/VBM/FED055', '/VBM/FED056', '/VBM/FED057', '/VBM/FED058', '/VBM/FED059', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED066', '/VBM/FED067', '/VBM/FED068']


In [4]:
# make dict with all T1-images as pathlike objects
images_T1={}
for i in subdirs:
    for j in os.listdir(i):
        match=re.match(r'(.*T1_MPRAGE.*\d{14,}.*.nii)', j)
        if match:
            images_T1[i]=os.path.join(i, match.group(0))

In [5]:
# control T1 list
print(images_T1)

{'/VBM/FED006': '/VBM/FED006/CON_FED006_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007': '/VBM/FED007/CON_FED007_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008': '/VBM/FED008/CON_FED008_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii', '/VBM/FED009': '/VBM/FED009/CON_FED009_T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954.nii', '/VBM/FED010': '/VBM/FED010/CON_FED010_T1_MPRAGE_SAG_ISO_0_9_0005_20141119091904.nii', '/VBM/FED011': '/VBM/FED011/DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED012': '/VBM/FED012/CON_FED012_T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148.nii', '/VBM/FED013': '/VBM/FED013/CON_FED013_T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955.nii', '/VBM/FED014': '/VBM/FED014/DEP_FED014_T1_MPRAGE_SAG_ISO_0_9_0005_20141125153216.nii', '/VBM/FED015': '/VBM/FED015/CON_FED015_T1_MPRAGE_SAG_ISO_0_9_0005_20141202145526.nii', '/VBM/FED016': '/VBM/FED016/CON_FED016_T1_MPRAGE_SAG_ISO_0_9_0005_20141202164821.nii', '/VBM/FED017': '/VBM/FED017/CON_FED017_T1_

Run NewSegment from SPM12 using tissue probabilities for substance classification

In [6]:
# use NewSegment from spm to segment unimportant anatomical landmarks and perform "brain-extraction" (TEST FED006)
#for i in images.keys(): use this when ready to segment full subject list

# classify tissues from tpm.nii
# need to perform segmentation with full tissue information, or brain will be classified as "soft tissue" etc. -> see examples in "/VBM/FED006/"
TPM_dir = '/opt/spm12-r7219/spm12_mcr/spm12/tpm/TPM.nii'
tissue1 = ((TPM_dir, 1), 2, (True,True), (False, False)) # GM
tissue2 = ((TPM_dir, 2), 2, (True,True), (False, False)) # WM
tissue3 = ((TPM_dir, 3), 2, (True,True), (False, False)) # CSF
tissue4 = ((TPM_dir, 4), 2, (False,False), (False, False)) # bone
tissue5 = ((TPM_dir, 5), 2, (False,False), (False, False)) # soft tissue
tissue6 = ((TPM_dir, 6), 2, (False,False), (False, False)) # air and background

# create SKULLSTRIP segment nodes for each subject
for k,v in images_T1.items():
    segm_skullstrip_T1=Node(spm.preprocess.NewSegment(), name='DARTEL_newsegment')
    segm_skullstrip_T1.inputs.tissues=[tissue1, tissue2, tissue3, tissue4, tissue5, tissue6]
    # do not generate SPM8 and higher compatible jobs
    segm_skullstrip_T1.inputs.use_v8struct=False
    segm_skullstrip_T1.inputs.affine_regularization='none'
    # write forward and inverse deformation fields
    #segm_skullstrip_T1.inputs.write_deformation_fields=[True, True]
    # establish input files from subject
    segm_skullstrip_T1.inputs.channel_files=v
    # establish basedir for output of subject node
    segm_skullstrip_T1.base_dir=k

In [ ]:
# run segm_skullstrip_T1 and print outputs
results=segm_skullstrip_T1.run()
print(results.outputs)

Gather requirements for DARTEL template creation and check their order (assumptions!)

In [6]:
# get segmentation directory for each subject

# define start of segmentation directory name
targetnode="segm"
taskdirs={}

for i in images_T1.keys():
    for j in subdirs:
        # match keys with subdir list
        if j == i: 
            for k in os.listdir(j):
                # go through content of subdirs and look for "true" directories
                if os.path.isdir(os.path.join(j, k)):
                    # regex functions match=exact match of pattern, search=finds elements that contain pattern
                    # regex matches entries in j with targetnode k
                    match=re.match(r'.*(' + targetnode + ').*', k)
                    # ? This is needed to recognize the attribute group from the regex output
                    if match: 
                        taskdirs[j]=os.path.join(j, match.group(0))

In [7]:
# split taskdirs in control and depressed

# define and load from .xls(x) files
sub_control_ID="FED_Subject_Assignments.xlsx"
control_ID=pd.read_excel(sub_control_ID, sheet_name="Sheet2").astype(str)
ID=control_ID['Sub Num FED_XXX'].str.zfill(3).tolist()
status=control_ID['BDI 22 Score'].tolist()


control_taskdir={}
depress_taskdir={}

for i in taskdirs.keys():
    for x,y in zip(ID,status):
        if i.rsplit('/',1)[1] == "FED"+x and y == "0":
            control_taskdir[i]=taskdirs[i]

for i in taskdirs.keys():
    if i not in control_taskdir.keys():
        depress_taskdir[i]=taskdirs[i]

In [8]:
# check dicts
print(control_taskdir.keys())
print(depress_taskdir.keys())
# see, if sizes are about equal
print(len(control_taskdir.keys()), len(depress_taskdir.keys()))

print(taskdirs.items(), "\n", len(taskdirs.items()))

dict_keys(['/VBM/FED006', '/VBM/FED007', '/VBM/FED008', '/VBM/FED009', '/VBM/FED010', '/VBM/FED012', '/VBM/FED013', '/VBM/FED015', '/VBM/FED016', '/VBM/FED017', '/VBM/FED019', '/VBM/FED020', '/VBM/FED021', '/VBM/FED022', '/VBM/FED023', '/VBM/FED024', '/VBM/FED025', '/VBM/FED026', '/VBM/FED049', '/VBM/FED050', '/VBM/FED053', '/VBM/FED054', '/VBM/FED056', '/VBM/FED058', '/VBM/FED060', '/VBM/FED061', '/VBM/FED062', '/VBM/FED063', '/VBM/FED064', '/VBM/FED065', '/VBM/FED067', '/VBM/FED068'])
dict_keys(['/VBM/FED011', '/VBM/FED014', '/VBM/FED018', '/VBM/FED027', '/VBM/FED028', '/VBM/FED029', '/VBM/FED030', '/VBM/FED031', '/VBM/FED032', '/VBM/FED033', '/VBM/FED034', '/VBM/FED035', '/VBM/FED036', '/VBM/FED037', '/VBM/FED038', '/VBM/FED039', '/VBM/FED040', '/VBM/FED041', '/VBM/FED042', '/VBM/FED043', '/VBM/FED044', '/VBM/FED045', '/VBM/FED046', '/VBM/FED047', '/VBM/FED048', '/VBM/FED051', '/VBM/FED052', '/VBM/FED055', '/VBM/FED057', '/VBM/FED059', '/VBM/FED066'])
32 31
dict_items([('/VBM/FED006

In [9]:
# create template dict

# only 31 depressed, so take all into template
template_dirs=depress_taskdir.copy()
# collect random keys (31 from control)
#keys=random.sample([control_taskdir.keys()], 31)

# for speed reasons use this short cut during de-bugging ELIMINATE WHEN DONE
keys=[i for i in control_taskdir.keys() if i != os.path.join(VBM_dir, "FED010")]

# integrate both dicts for template_dirs
for i in keys:
    template_dirs[i]=control_taskdir[i]

# get rc1 and rc2 files as path-like objects
rc1=[]
rc2=[]
rc3=[]
# append files to lists
for k,v in template_dirs.items():
    for i in os.listdir(v):
        # look for rc1(GM)
        match1=re.match(r'(^rc1.*.nii)', i)
        if match1:
            rc1.append(os.path.join(v, match1.group(0)))
        # look for rc2(WM)
        match2=re.match(r'(^rc2.*.nii)', i)
        if match2:
            rc2.append(os.path.join(v, match2.group(0)))
        # look for rc3(CSF)
        match3=re.match(r'(^rc3.*.nii)', i)
        if match3:
            rc3.append(os.path.join(v, match3.group(0)))
# sort lists
rc1=sorted(rc1)
rc2=sorted(rc2)
rc3=sorted(rc3)

In [10]:
# compare contrcontrol_taskdir and template_dirs to determined the omitted subject
omitted=set(control_taskdir.keys()) - set(template_dirs.keys())
# print excluded FED subject
print("The excluded FED subject is:   ", omitted)

# print lists
print(rc1[:7], "\n", len(rc1))
print(rc2[:7], "\n", len(rc2))
print(rc3[:7], "\n", len(rc3))

The excluded FED subject is:    {'/VBM/FED010'}
['/VBM/FED006/DARTEL_newsegment/rc1CON_FED006_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007/DARTEL_newsegment/rc1CON_FED007_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008/DARTEL_newsegment/rc1CON_FED008_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii', '/VBM/FED009/DARTEL_newsegment/rc1CON_FED009_T1_MPRAGE_SAG_ISO_0_9_0005_20141117154954.nii', '/VBM/FED011/DARTEL_newsegment/rc1DEP_FED011_T1_MPRAGE_SAG_ISO_0_9_0005_20141119161708.nii', '/VBM/FED012/DARTEL_newsegment/rc1CON_FED012_T1_MPRAGE_SAG_ISO_0_9_0005_20141124150148.nii', '/VBM/FED013/DARTEL_newsegment/rc1CON_FED013_T1_MPRAGE_SAG_ISO_0_9_0005_20141125093955.nii'] 
 62
['/VBM/FED006/DARTEL_newsegment/rc2CON_FED006_T1_MPRAGE_SAG_ISO_0_9_0005_20141106153429.nii', '/VBM/FED007/DARTEL_newsegment/rc2CON_FED007_T1_MPRAGE_SAG_ISO_0_9_0005_20141107104226.nii', '/VBM/FED008/DARTEL_newsegment/rc2CON_FED008_T1_MPRAGE_SAG_ISO_0_9_0005_20141112155009.nii', '/VBM/FED009/DA

In [11]:
# check that lists are both in the same subject order and template generation will not make wrong assumptions ...^^
for x,y,z in zip_longest(rc1,rc2,rc3):
    if x.rsplit('/', 3)[1] ==  y.rsplit('/', 3)[1] and x.rsplit('/', 3)[1] ==  z.rsplit('/', 3)[1]:
        print("Yes!")
        print(x.rsplit('/', 3)[1],"  is the same as  ", y.rsplit('/', 3)[1], " and ", z.rsplit('/', 3)[1])
    else:
        print("Noooooooooooooooooooooooooooooooooo")
        print(x.rsplit('/', 3)[1],"  is neither  ", y.rsplit('/', 3)[1], " nor ", z.rsplit('/', 3)[1])

Yes!
FED006   is the same as   FED006  and  FED006
Yes!
FED007   is the same as   FED007  and  FED007
Yes!
FED008   is the same as   FED008  and  FED008
Yes!
FED009   is the same as   FED009  and  FED009
Yes!
FED011   is the same as   FED011  and  FED011
Yes!
FED012   is the same as   FED012  and  FED012
Yes!
FED013   is the same as   FED013  and  FED013
Yes!
FED014   is the same as   FED014  and  FED014
Yes!
FED015   is the same as   FED015  and  FED015
Yes!
FED016   is the same as   FED016  and  FED016
Yes!
FED017   is the same as   FED017  and  FED017
Yes!
FED018   is the same as   FED018  and  FED018
Yes!
FED019   is the same as   FED019  and  FED019
Yes!
FED020   is the same as   FED020  and  FED020
Yes!
FED021   is the same as   FED021  and  FED021
Yes!
FED022   is the same as   FED022  and  FED022
Yes!
FED023   is the same as   FED023  and  FED023
Yes!
FED024   is the same as   FED024  and  FED024
Yes!
FED025   is the same as   FED025  and  FED025
Yes!
FED026   is the same as   

create GM template using SPM DARTEL

In [12]:
# create DARTEL node and template
dartel_template = Node(spm.DARTEL(), name='DARTEL_template')
dartel_template.inputs.image_files = [rc1, rc2, rc3] # simultaneously register/warp GM, WM and CSF together
dartel_template.inputs.template_prefix = "template"
dartel_template.base_dir = VBM_dir
# do not generate SPM8 and higher compatible jobs
dartel_template.inputs.use_v8struct = False

In [ ]:
# THINK ABOUT INCLUDING CSF FROM HERE ON!!! -> IMPLICATIONS FOR DEFORMATION FIELDS (accuracy, specificity etc)???

Gather requirements for DARTEL Normalisation to MNI space and check their order (assumptions!)

In [ ]:
# run dartel_template and print outputs
results=dartel_template.run()
print(results.outputs)

In [13]:
# get required file lists and check that theyre in the same order

# template dir
finaltemplate_dir= os.path.join(dartel_template.base_dir, dartel_template.name)

# flowfields
flowfields = [i for i in os.listdir(finaltemplate_dir) if re.match(r'(^u_rc1.*.nii)', i)]
# convert flowfield niftis into full-path objects
flowfields = [os.path.join(finaltemplate_dir, i) for i in flowfields]

# GM, WM and CSF (for Quality control and normalisation)
c1 = []
c2 = []
c3 = []

# get c1/2/3 dirs from template-subjects
intemplate_dirs={}
for i in flowfields:
    for k,v in taskdirs.items():
        if i.split('_',4)[3] == k.rsplit('/',1)[1]:
            intemplate_dirs[v]=i

# fill c1, c2 and c3 lists
# look for c1(GM)
c1 = [os.path.join(k, i) for k,v in intemplate_dirs.items() for i in os.listdir(k) if re.match(r'(^c1.*.nii)', i)]
# look for c2(WM)
c2 = [os.path.join(k, i) for k,v in intemplate_dirs.items() for i in os.listdir(k) if re.match(r'(^c2.*.nii)', i)]
# look for c3(CSF)
c3 = [os.path.join(k, i) for k,v in intemplate_dirs.items() for i in os.listdir(k) if re.match(r'(^c3.*.nii)', i)]

In [14]:
# check flowfields, c1 , c2, c3
print(flowfields[0:4], "\n")
print(c1[0:4], "\n")
print(c2[0:4], "\n")
print(c3[0:4], "\n")
# check order -> avoid wrong assumptions in normalisation^^
for w,x,y,z in zip_longest(flowfields,c1,c2,c3):
    if w.split('_', 4)[3] ==  x.split('_', 3)[2] and y.split('_', 3)[2] and z.split('_', 3)[2]:
        print("Yes!")
        print(w.split('_', 4)[3], "  is the same as  ",x.split('_', 3)[2], y.split('_', 3)[2], "and",z.split('_', 3)[2])
    else:
        print("Nope! Have another look ... ")
        print(w.split('_', 4)[3], "  is not  ", x.split('_', 3)[2], y.split('_', 3)[2], "or", z.split('_', 3)[2])

['/VBM/DARTEL_template/u_rc1CON_FED049_T1_MPRAGE_SAG_ISO_0_9_0009_20151123115353_template.nii', '/VBM/DARTEL_template/u_rc1DEP_FED059_T1_MPRAGE_SAG_ISO_0_9_0005_20160218100702_template.nii', '/VBM/DARTEL_template/u_rc1DEP_FED033_T1_MPRAGE_SAG_ISO_0_9_0005_20150326151309_template.nii', '/VBM/DARTEL_template/u_rc1DEP_FED035_T1_MPRAGE_SAG_ISO_0_9_0005_20150917120306_template.nii'] 

['/VBM/FED049/DARTEL_newsegment/c1CON_FED049_T1_MPRAGE_SAG_ISO_0_9_0009_20151123115353.nii', '/VBM/FED059/DARTEL_newsegment/c1DEP_FED059_T1_MPRAGE_SAG_ISO_0_9_0005_20160218100702.nii', '/VBM/FED033/DARTEL_newsegment/c1DEP_FED033_T1_MPRAGE_SAG_ISO_0_9_0005_20150326151309.nii', '/VBM/FED035/DARTEL_newsegment/c1DEP_FED035_T1_MPRAGE_SAG_ISO_0_9_0005_20150917120306.nii'] 

['/VBM/FED049/DARTEL_newsegment/c2CON_FED049_T1_MPRAGE_SAG_ISO_0_9_0009_20151123115353.nii', '/VBM/FED059/DARTEL_newsegment/c2DEP_FED059_T1_MPRAGE_SAG_ISO_0_9_0005_20160218100702.nii', '/VBM/FED033/DARTEL_newsegment/c2DEP_FED033_T1_MPRAGE_SAG_ISO

In [15]:
# grouptemplate_6 has two volumes!!!!!! GM & WM -> seperate them
# get template 
grouptemplate = os.path.join(finaltemplate_dir, dartel_template.inputs.template_prefix)

In [ ]:
# convert to nb object
grouptemplate_nb = nb.load(grouptemplate + "_6.nii")

# get seperate GM and WM volumes
grouptemplate_GM_WM_CSF = nb.funcs.four_to_three(grouptemplate_nb)

# save each volume as nifti file
grouptemplate_GM_WM_CSF[0].to_filename(grouptemplate + "_6_GM.nii")
grouptemplate_GM_WM_CSF[1].to_filename(grouptemplate + "_6_WM.nii")
grouptemplate_GM_WM_CSF[2].to_filename(grouptemplate + "_6_CSF.nii")

normalise to MNI space

In [16]:
# create DARTEL node to norm2MNI for GM and WM
tissues={"c1": ["_6_GM.nii",c1],  "c2": ["_6_GM.nii",c2]} # add:, "c3": ["_6_CSF.nii",c3]
for k,v in tissues.items():
    dartel_norm2mni=Node(spm.DARTELNorm2MNI(), name='DARTEL_norm2mni_' + k)
    dartel_norm2mni.inputs.template_file = grouptemplate + v[0]
    dartel_norm2mni.inputs.flowfield_files = flowfields # using same flowfields here creates overblown WM when normalising to WM template!
    dartel_norm2mni.inputs.apply_to_files = v[1]
    # "modulate (a boolean) – Modulate out images - no modulation preserves concentrations."(nipype documentation)
    dartel_norm2mni.inputs.modulate = True
    dartel_norm2mni.base_dir = VBM_dir
    # think about smoothing kernel below: btw 4mm and 12mm -> ROI sizes?
    dartel_norm2mni.inputs.fwhm = 10
    # do not generate SPM8 and higher compatible jobs
    dartel_norm2mni.inputs.use_v8struct = False
    # run normalisation and print outputs
    results = dartel_norm2mni.run()
    print(results.outputs)

200303-21:54:32,240 nipype.workflow INFO:
	 [Node] Setting-up "DARTEL_norm2mni_c1" in "/VBM/DARTEL_norm2mni_c1".
200303-21:54:32,267 nipype.workflow INFO:
	 [Node] Running "DARTEL_norm2mni_c1" ("nipype.interfaces.spm.preprocess.DARTELNorm2MNI")
200303-22:00:46,582 nipype.workflow INFO:
	 [Node] Finished "DARTEL_norm2mni_c1".

normalization_parameter_file = /VBM/DARTEL_norm2mni_c1/template_6_GM_2mni.mat
normalized_files = ['/VBM/DARTEL_norm2mni_c1/smwc1CON_FED049_T1_MPRAGE_SAG_ISO_0_9_0009_20151123115353.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED059_T1_MPRAGE_SAG_ISO_0_9_0005_20160218100702.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED033_T1_MPRAGE_SAG_ISO_0_9_0005_20150326151309.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED035_T1_MPRAGE_SAG_ISO_0_9_0005_20150917120306.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_FED065_T1_MPRAGE_SAG_ISO_0_9_0005_20160310090712.nii', '/VBM/DARTEL_norm2mni_c1/smwc1CON_FED058_T1_MPRAGE_SAG_ISO_0_9_0005_20160217151251.nii', '/VBM/DARTEL_norm2mni_c1/smwc1DEP_FED052

Order and Quality Control:

In [17]:
# get normalised GM/WM images to analyse in MultipleRegression model
# change norm_dir to vary between GM and WM analysis
norm_dirs = [os.path.join(VBM_dir, "%s") %(i) for i in os.listdir(VBM_dir) if i.startswith("DARTEL_norm2mni")]

# grep files according to regex
images_mreg_GM=[i for i in os.listdir(norm_dirs[0]) if re.match(r'(smwc.*\d{14,}.*.nii)', i)]
images_mreg_WM=[i for i in os.listdir(norm_dirs[1]) if re.match(r'(smwc.*\d{14,}.*.nii)', i)]

# turn filenames into full-path objects
images_mreg_GM=[os.path.join(norm_dirs[0], i) for i in images_mreg_GM]
images_mreg_WM=[os.path.join(norm_dirs[1], i) for i in images_mreg_WM]

In [18]:
# create 4D control volume for visual inspection
for dirs,files in zip(norm_dirs,[images_mreg_GM,images_mreg_WM]):
    # create 4D control-volume of all smwc1/2 images
    FED_norm2MNI_control = nb.funcs.concat_images(files, check_affines=True, axis=None)
    
    # save the files (gzipped!)
    FED_norm2MNI_control.to_filename(os.path.join(dirs, "FED_smw" + dirs[-2:] + "_4D_control.nii.gz"))
   
    # un-cache
    FED_norm2MNI_control.uncache()
    
    
# -> also do this for Template images (u_rc1) and all segmented images (c1-3)

# same for the flowfields created by the DARTEL template algorithm
#FED_templateflowfields_control = nb.funcs.concat_images(flowfields, check_affines=False, axis=None)
#FED_templateflowfields_control.to_filename(os.path.join(norm_dir, "FED_u_rc1-flowfields_4D_control.nii.gz"))
#FED_templateflowfields_control.uncache()

# use c1 images first (too little space on local)
#FED_c1_control = nb.funcs.concat_images(c1, check_affines=False, axis=None)
#FED_c1_control.to_filename(os.path.join(norm_dir, "FED_c1_4D_control.nii.gz"))
#FED_c1_control.to_filename(os.path.join(norm_dir, "FED_c1_4D_control.nii.gz"))

#FED_c2_control = nb.funcs.concat_images(c2, check_affines=False, axis=None)

# save the files (gzipped!)
#FED_c2_control.to_filename(os.path.join(norm_dir, "FED_c2_4D_control.nii.gz"))

# un-cache
#FED_c2_control.uncache()

#FED_c3_control = nb.funcs.concat_images(c3, check_affines=False, axis=None)

# save the files (gzipped!)
#FED_c3_control.to_filename(os.path.join(norm_dir, "FED_c3_4D_control.nii.gz"))

# un-cache
#FED_c3_control.uncache()

# -> view fsleyes -movie-mode to select possible errors